In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Generate time series data
def generate_time_series_data(n_steps):
    time = np.linspace(0, 400, n_steps)
    signal = (2 * np.sin(0.2 * time)  # Slow oscillation
              + 0.5 * np.cos(0.8 * time)  # Fast oscillation
              + 0.01 * time  # Linear trend
              + np.random.normal(scale=0.2, size=time.shape))  # Random noise

    return signal, time

# Create the LSTM model
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(30, input_shape=input_shape, return_sequences=False),
        Dense(8, activation='relu'),
        Dense(1)
    ])
    return model

# ABC Approximation Function
def approximate_bayesian_computation(model, observed_data, n_samples=1000, tol=0.05):
    parameters = []
    simulated_errors = []

    for _ in range(n_samples):
        # Randomly initialize model weights
        for layer in model.layers:
            if isinstance(layer, LSTM) or isinstance(layer, Dense):
                weights = [tf.random.normal(w.shape) for w in layer.get_weights()]
                layer.set_weights(weights)

        # Simulate data using the model
        simulated_data = model.predict(observed_data[0])

        # Calculate error between observed and simulated data
        error = np.mean((simulated_data - observed_data[1])**2)

        if error < tol:
            parameters.append([layer.get_weights() for layer in model.layers if isinstance(layer, (LSTM, Dense))])
            simulated_errors.append(error)

    return parameters, simulated_errors

# Prepare training data
n_steps = 1000
signal, time = generate_time_series_data(n_steps)

# Reshape data for LSTM input
input_data = signal[:-1].reshape((1, n_steps - 1, 1))
target_data = signal[1:].reshape((n_steps - 1, 1))

# Initialize model
input_shape = (n_steps - 1, 1)
model = create_lstm_model(input_shape)

# Perform ABC optimization
parameters, errors = approximate_bayesian_computation(model, (input_data, target_data), n_samples=1000, tol=0.2)  # Increased tolerance

# Ensure there are results
    # Select best parameters based on error
best_index = np.argmin(errors)
best_parameters = parameters[best_index]

# Assign best parameters to model
for layer, best_layer_weights in zip(model.layers, best_parameters):
    if isinstance(layer, LSTM) or isinstance(layer, Dense):
        layer.set_weights(best_layer_weights)

# Evaluate the model
predictions = model.predict(input_data)
print("Predictions:", predictions)

# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(time[1:], target_data.flatten(), label="Actual Signal", color="blue")
plt.plot(time[1:], predictions.flatten(), label="Predicted Signal", color="red", linestyle="--")
plt.xlabel("Time")
plt.ylabel("Signal Value")
plt.title("Actual vs Predicted Signal")
plt.legend()
plt.show()


1/1 [==============================] - 0s 59ms/step


ValueError: attempt to get argmin of an empty sequence